Valko Danila 11/07/21

Решение выполнено в первом приближении: обрабатывает только общий формат отчёта, извлекает только одну таблицу из каждого отчёта и агрегирует. Без NLP и прочей магии.


Содержит блоки:
- Скрапинг файлов с сайта Росгидромета
- Конвертер в docx
- Парсинг отчётов
- Нормализация топонимов и прочее
- Матчинг координат и просмотр данных в Leaflet карте


---



## Скрапинг файлов с сайта Росгидромета 
http://www.meteorf.ru/product/infomaterials/99/?year=2021&ID=99


На входе url сайта. По итогу вычисления блока получаем каталог files (126 шт.), в котором находятся файлы отчёта вида yyyy.mm.ext и zip архив с ними.

In [ ]:
!pip install beautifulsoup4==4.9.1
from bs4 import BeautifulSoup as bs
!pip install requests
import requests
import calendar
import os

In [ ]:
def get_month(s):
  ru_month = ['январ', 'феврал', 'март', 'апрел', 'мае', 'июн', 'июл', 'август', 'сентябр', 'октябр', 'ноябр', 'декабр']
  s = str.lower(s)
  for i in ru_month:
    if s.find(i) > -1:
      return ru_month.index(i)+1
  return 0    


In [ ]:
def get_links(fr = 2008, to = 2021):
  links = []
  for y in range(fr, to+1):
    res = requests.get("http://www.meteorf.ru/product/infomaterials/99/?year="+str(y))
    soup = bs(res.content)
    items = soup.select("div.documents.bottom-indent > div > div.item.item_y"+str(y))
    for i in items:
      m = get_month(str(i.contents))
      l = "http://www.meteorf.ru"+str(i.find("a")['href'])
      links.append([str(y)+"."+str(m).zfill(2), l])
  return links

In [ ]:
#items = soup.select("div.documents.bottom-indent > div > div.item.item_y2021")
#div.documents.bottom-indent > div > div:nth-child(1) > div.rcol > ul > li > a
#div.documents.bottom-indent > div > div:nth-child(2) > div.rcol > ul > li > a
#items[0].find("a")['href']
#http://www.meteorf.ru/upload/iblock/749/2021-1%80%D0%B0%D1%86%D0%B8%D0%B8.docx

In [ ]:
def download_files(links):
  for i in links:
    if i != None:
      f = requests.get(i[1], allow_redirects=True)
      open("files/"+i[0]+os.path.splitext(i[1])[1], 'wb').write(f.content)

In [ ]:
links = get_links()

In [ ]:
#Ручные корректировки
#12.2012 - есть дубликат декабрьского -links[29]
#7.2013 - приложения отдельно лежат +links[35] -links[36:38]
#5.2018 - веб-страничка вместо файла ~links[97]
#2.2018 - частичный отчёт -links[101]
#12.2019 - декабрь 2018 попал на страницу 2019 -links[115]
#9.2020 - частичный отчёт -links[120]
links[29] = None
links[36] = None
links[37] = None
links[38] = None
links[97][0] = links[97][0]+".html"
links[101] = None
links[115] = None
links[120] = None

In [ ]:
links[:5]

[['2008.04', 'http://www.meteorf.ru/upload/iblock/72b/mes04r.doc'],
 ['2008.09', 'http://www.meteorf.ru/upload/iblock/e89/av-sent.doc'],
 ['2008.11', 'http://www.meteorf.ru/upload/iblock/d27/av-noy.doc'],
 ['2008.12', 'http://www.meteorf.ru/upload/iblock/a0a/av-dec.doc'],
 ['2009.01', 'http://www.meteorf.ru/upload/iblock/52e/av-r-yanv.doc']]

In [ ]:
!rm -r "./files/"
!mkdir "./files/"
download_files(links)

In [ ]:
!zip -r files.zip files

## Конвертер в docx


На входе каталог files с файлами отчётов в форматах doc и html. По итогу вычисления блока получаем каталог files с файлами (126 шт.) в формате docx и zip архив с ними

In [ ]:
!sudo add-apt-repository ppa:libreoffice/ppa && sudo apt-get update

In [ ]:
!sudo apt-get install unoconv

In [ ]:
!unoconv -f docx files/*.doc

In [ ]:
!unoconv -f docx files/*.html

In [ ]:
!rm files/*.doc

In [ ]:
!rm files/*.html

In [ ]:
!zip -r files_docx.zip files

## Парсинг отчётов
На фходе каталог files с файлами отчётов в формате docx. Файлы закачаны скриптами выше: https://drive.google.com/file/d/1hXpp6QCr6OKgH2m65Fa7BaCvDkTsrMlx/view?usp=sharing


По итогу вычисления блока получаем pandas переменную data с талицей приложения 1 из отчётов "Перечень случаев 
экстремально высокого загрязнения поверхностных вод суши".


Результат сохранен в data.csv
https://drive.google.com/file/d/1ZgRPH4dCk4_46O9JbNOf1hRwdzVNluZp/view?usp=sharing

In [ ]:
import os
ls = os.listdir("files")
ls = sorted(ls)[:22]
ls[:4]

In [ ]:
!git clone https://github.com/CAG-ru/cag-public

In [ ]:
%%capture
!pip install mammoth
!pip install xlsx2html
!pip install xls2xlsx
!pip install pandas

In [ ]:
%%capture
os.chdir("cag-public/lib/report_parser/")
!python3 report.py

In [ ]:
import pandas as pd
from report import Report, ConfigParser
config = ConfigParser('configfile.json') 

In [ ]:
os.chdir("../../../")
!ls

In [ ]:
def tidy_table(t):
  #Вытащим пока то, что нормально вытаскивается
  p = t[t[0].str.isnumeric()]
  p[4] = pd.to_numeric(p[4], errors='coerce')
  p = p.dropna()
  return p

In [ ]:
%%capture
#Распарсим, что распарсится
tabs = []
for i in ls: 
  try:
    rep = Report("files/"+i, config)
    rep.parse()
    tab = rep.find_table_by_name('Приложение')[0]
    tab = tidy_table(tab)
    tab['y'] = i.split('.')[0]
    tab['m'] = i.split('.')[1]
    tabs.append(tab)
  except:
    0

In [ ]:
data = pd.concat(tabs)

In [ ]:
data.to_csv("data.csv")

In [ ]:
data.head(3) #Финальные данные из отчётов

In [ ]:
#example = 'files/2008.11.docx'
#report = Report(example, config)
#report.parse()
#table = report.find_table_by_name('Приложение')[0]
#table[:4]

In [ ]:
#p = table[table[0].str.isnumeric()]
#p[4] = pd.to_numeric(p[4], errors='coerce')
#p = p.dropna()
#p

## Нормализация топонимов и прочее

(Предполагаю, что в задании имеется ввиду разбор муниципалитетов, а не регионов, но это очень хлопотно. Поэтому ниже пойдёт решение по субъектам РФ, с точки зрения демонстрации навков разница не большая)

На входе pandas таблица data.

На выходе pandas таблица ndata, с нормализованнми наименованиями субъектов РФ в колонке loc, y - год, m - месяц, p -загрязнитель, v - превышение раз ПДК

Результат сохранен в ndata.csv https://drive.google.com/file/d/1tjES-lHKP_2XGK4BAV6Vbn994ED9IRtJ/view?usp=sharing

In [ ]:
os.chdir("../../../")
!ls

In [ ]:
%%capture
!pip install fuzzywuzzy
!pip install requests

In [ ]:
%%capture
os.chdir("cag-public/lib/")
!python3 normalization/normalize_locations.py

In [ ]:
from normalization.normalize_locations import *

In [ ]:
normalize_location_name('  Кемировская область')
#"Кемеровская область – Кузбасс"

In [ ]:
os.chdir("./content")
!ls

In [ ]:
data = pd.read_csv("./data.csv")
data.rename({data.columns[2] : 'obj', data.columns[3] : 'loc', data.columns[4] : 'p', data.columns[5] : 'v'}, axis=1, inplace = True)
data.drop(data.columns[[0, 1]], axis = 1, inplace = True) #два ненужных столбца с нумерацией
data.head(3)

,obj,loc,p,v,y,m
0,"р. Айва, г.Красноуральск",Свердловская область,Ионы меди,60.0,2008,11
1,"р. Колос-Йоки, | п. Никель",Мурманская область,Ионы никеля,55.0,2008,11
2,"р. Нюдуай, | г. Мончегорск",Мурманская область,Ионы меди,107.0,2008,11


In [ ]:
len(data)

416

In [ ]:
data['loc']=data['loc'].apply(lambda x: normalize_location_name(x))
#выполнялось пять минут О_о

In [ ]:
#просто попробуем разобрать колонку с реками и городами
#data['obj']=data[data.columns[0]].apply(lambda x: normalize_location_name(x))
#ещё 5 минут..

In [ ]:
data

,obj,loc,p,v,y,m
0,"р. Айва, г.Красноуральск",Свердловская область,Ионы меди,60.0,2008,11
1,"р. Колос-Йоки, | п. Никель",Мурманская область,Ионы никеля,55.0,2008,11
2,"р. Нюдуай, | г. Мончегорск",Мурманская область,Ионы меди,107.0,2008,11
3,"р. Охинка, г. Оха",Сахалинская область,Нефтепродукты,61.0,2008,11
4,"р. Айва, г.Красноуральск",Свердловская область,Ионы марганца,61.0,2008,11
...,...,...,...,...,...,...
411,"р. Тобол, г. Курган",Курганская область,Ионы марганца,72.0,2011,9
412,"р. Тула, | г. Новосибирск",Новосибирская область,Ионы марганца,198.0,2011,9
413,"р. Упа, г. Тула",Тульская область,Азот нитритный,79.0,2011,9
414,"р.Камышенка, | г. Новосибирск",Новосибирская область,Ионы марганца,63.0,2011,9


In [ ]:
data.to_csv("ndata.csv", index=False)

## Матчинг координат


На входе pandas таблица ndata

На выходе таблица cdata  с доп. колонками lat и lon сматченными по loc

Результат сохранен в cdata.csv https://drive.google.com/file/d/17MQ_ab1mCkGeYy-Z1xeiC4LNZbQ0B50-/view?usp=sharing

In [ ]:
ndata = pd.read_csv("./ndata.csv")

In [ ]:
ndata.head(3)

,obj,loc,p,v,y,m
0,"р. Айва, г.Красноуральск",Свердловская область,Ионы меди,60.0,2008,11
1,"р. Колос-Йоки, | п. Никель",Мурманская область,Ионы никеля,55.0,2008,11
2,"р. Нюдуай, | г. Мончегорск",Мурманская область,Ионы меди,107.0,2008,11


In [ ]:
!git clone https://gist.github.com/max107/6571147
#нашёл какие-то коррдинаты регионов

In [ ]:
coord = pd.read_json ("6571147/Regions.json")

In [ ]:
coord = coord[:1].T

In [ ]:
coord = coord.rename_axis("loc").reset_index()


In [ ]:
coord['loc'] = coord['loc'].apply(lambda x: normalize_location_name(x))

In [ ]:
#типа геометрические центры регионов
coord['lat'] = coord[coord.columns[1]].apply(lambda x: pd.Series(v[0] for v in x).mean())
coord['lon'] = coord[coord.columns[1]].apply(lambda x: pd.Series(v[1] for v in x).mean())

In [ ]:
coord.drop(coord.columns[1], axis = 1, inplace = True)
coord

,loc,lat,lon
0,Алтайский край,52.735033,82.948320
1,Республика Алтай,50.833761,87.215391
2,Курская область,51.672606,36.071685
3,Тульская область,53.948215,37.609363
4,Пермский край,58.296433,56.226613
...,...,...,...
76,Камчатский край,59.557543,164.334764
77,Ненецкий автономный округ,68.141823,53.597795
78,Ямало-Ненецкий автономный округ,68.143527,74.749404
79,Краснодарский край,45.285264,39.515898


In [ ]:
#join
cdata = pd.merge(ndata, coord, on="loc")

In [ ]:
cdata

,obj,loc,p,v,y,m,lat,lon
0,"р. Айва, г.Красноуральск",Свердловская область,Ионы меди,60.0,2008,11,58.144054,61.078888
1,"р. Айва, г.Красноуральск",Свердловская область,Ионы марганца,61.0,2008,11,58.144054,61.078888
2,"р. Пышма, г. Талица",Свердловская область,Взвешенные вещества,53.0,2008,11,58.144054,61.078888
3,"вдхр. Верхне-Исетское, в районе водозабора г.Е...",Свердловская область,Взвешенные вещества,261.0,2008,12,58.144054,61.078888
4,"р. Исеть, г. Екатеринбург",Свердловская область,Взвешенные вещества,106.0,2008,12,58.144054,61.078888
...,...,...,...,...,...,...,...,...
407,"р. Камбилеевка, | с. Камбилеевское",Республика Северная Осетия – Алания,Легкоокисляемые органические вещества по | БП...,48.0,2011,9,43.226666,44.323080
408,"р. Правая Хетта, | пос. Пангоды",Ямало-Ненецкий автономный округ,Ионы цинка,91.0,2010,10,68.143527,74.749404
409,"р. Озерная, | п. Шумный",Камчатский край,Ионы железа общего,81.0,2011,9,59.557543,164.334764
410,"р. Паужетка, | п. Паужетка",Камчатский край,Ионы железа общего,80.0,2011,9,59.557543,164.334764


In [ ]:
cdata.to_csv("cdata.csv", index=False)


## Просмотр на карте

(Предполагаю, что в задании имеется ввиду разбор муниципалитетов, а не регионов, но это очень хлопотно. Поэтому ниже пойдёт решение по субъектам РФ, с точки зрения демонстрации навков разница не большая)

In [ ]:
import folium
from folium.plugins import MarkerCluster #Просто посмотрим в Leaflet
import pandas as pd

In [ ]:
# Без интерактива :) Посмотрим Ионы марганца в 2010...
d = cdata
d = d[d['y'] == 2010]
d = d[d['p'] == 'Ионы марганца']

In [ ]:
center = [d['lat'].iloc[0], d['lon'].iloc[0]] #Define coordinates of where we want to center our map
my_map = folium.Map(location = center, zoom_start = 3)

for i in d.itertuples():
  c = [i.lat, i.lon] #Add markers
  folium.RegularPolygonMarker(c, popup = '{}, {}: {} ПДК'.format(i.loc,i.p,i.v), fill_color = '#00ff40',
                            number_of_sides = 3, radius = 10).add_to(my_map)

#Display the map
my_map
#Если кликнуть на маркер, будет popup вида "Пермский край, Ионы марганца: 113.0 ПДК"


In [ ]:
cdata = pd.read_csv("./cdata.csv")

## Вариант с Shiny App (на скорую руку)
https://ellariel.shinyapps.io/region/